# Conception data for Experiment 1

In [7]:
# importation of the packages

import numpy as np
import random
import matplotlib.pyplot as plt
import pickle
import matplotlib.colors as colors
import matplotlib
import copy
from matplotlib.lines import Line2D 
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr
import pandas as pd
%matplotlib inline
import seaborn as sns
from scipy.stats import gaussian_kde
from collections import Counter
import math
import scipy.stats
import collections
import matplotlib.ticker as mtick
from matplotlib.ticker import FormatStrFormatter
from scipy.stats import norm 
from sklearn.neighbors import KernelDensity 
from sklearn.utils.fixes import parse_version 
from sklearn.svm import SVC # "Support vector classifier"
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error, r2_score
from collections import Counter
#from tabulate import tabulate
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
from multiprocessing import Pool
from  matplotlib.colors import LinearSegmentedColormap
from scipy.stats.stats import pearsonr 
from collections import defaultdict
import itertools
from scipy.stats import binom
import networkx as nx
import os

## A - Useful functions

### 1-Function to compute a random degree to an ego-node

In [8]:
def get_degree(N, mean_degree):
    
    # we cannot have a degree = 0 because otherwise the node will never become infected
    mean_degree = 4
    E = mean_degree * N / 2
    max_number_edges = N * (N-1) / 2
    p = E / max_number_edges
    degree = np.random.binomial(N, p)
    while degree == 0:
        degree = np.random.binomial(N, p)
    return(degree)

### 2-Function to compute the propagation

In [9]:
def make_propagation(contagion, beta, phi, N, mean_degree, rate_random, rate_neighbors, nber_sample):
    
    """
    This function compute a synthetic propagation for Experiment 1
    
    Parameters: 
    - contagion: 0 for simple, 1 for complex
    - beta: value of beta (if contagion=1, write anything)
    - phi: value of phi (if contagion=0, write anything)
    - N: number of the nodes
    - mean_degree: mean of the degree
    - rate_random: rate of infection by spontaneous adoption
    - rate_neighbors: rate of infection of the neighbours of the ego by spontaneous adoption
    - nber_sample: number of realisations needed
    
    Return:
    - The data base df, i.e., all the information about the contagion of the ego-node
    
    """
    
    df = pd.DataFrame(columns=['degree','nber_infected_neighbors','prop_infected_neighbors','sum_stimuli','nber_stimuli_by_neighbor','parameter','contagion', 'trajectory_node', 'trajectory_neighbors'])

    for it in range(nber_sample):

        # get a degree
        degree = get_degree(N, mean_degree)

        # get a list of neighbors
        set_neighbors = {k for k in range(degree)}

        # get list of infected neighbors
        set_infected_neighbors = set()

        # get list of infected neighbors
        set_susceptible_neighbors = {k for k in range(degree)}

        # initialisation
        state_ego = 'S'
        is_infected = False
        nber_stimuli = 0
        T_nei = np.zeros((degree,1))
        T_ego = [0]

        t = 0

        while state_ego == 'S':
            
            #try to infect the ego with the infection of the neighbors at that time
            # if the node is simple, we try to infect it via simple contagion
            if contagion == 0:
                parameter = beta
                is_infected = [random.random()<beta for k in range(len(set_infected_neighbors))]
                is_infected = any(is_infected)

            # if the node is complex, we try to infect it via complex contagion
            if contagion == 1:
                parameter = phi
                if degree != 0:
                    if (len(set_infected_neighbors)/degree) >= phi:
                        is_infected = True

            if (is_infected):
                state_ego = 'I'
                T_ego.append(1)
            else:
                T_ego.append(0)
                
            # during this time step the ego has received some stimuli
            nber_stimuli += len(set_infected_neighbors)
                
            #####################################

            # try to infect the neighbors
            nei_to_infect = []
            for nei in set_susceptible_neighbors:
                r = random.random()
                if r<=rate_neighbors:
                    nei_to_infect.append(nei)
            for nei in nei_to_infect: 
                set_susceptible_neighbors.remove(nei)
                set_infected_neighbors.add(nei)
                
            state_nei = [0 if nei in set_susceptible_neighbors else 1 for nei in set_neighbors]
            state_nei = np.asarray(state_nei).reshape((degree,1))
            T_nei = np.concatenate((T_nei, state_nei), axis=1)

        nber_infected_neighbors = len(set_infected_neighbors)
        nber_stimuli = nber_stimuli

        if degree != 0:
            prop_infected_neighbors = nber_infected_neighbors / degree
            nber_stimuli_by_neighbor = nber_stimuli / degree
        else:
            prop_infected_neighbors = 0
            nber_stimuli_by_neighbor = 0

        type_contagion = contagion

        new_row = {'degree': degree,
                  'nber_infected_neighbors': nber_infected_neighbors,
                   'prop_infected_neighbors': prop_infected_neighbors,
                   'sum_stimuli': nber_stimuli,
                   'nber_stimuli_by_neighbor': nber_stimuli_by_neighbor,
                   'parameter': parameter,
                   'contagion': type_contagion,
                   'trajectory_node': [T_ego],
                   'trajectory_neighbors': [T_nei]
                  }

        #df = df.append(new_row, ignore_index=True)
        df = pd.concat([df, pd.DataFrame(new_row)], ignore_index=True)

    type_contagion = contagion

    with open('df_experiment1/df_'+str(contagion)+'_'+str(parameter)+'_star_without_sp.pickle', 'wb') as handle:
        pickle.dump(df, handle)
        
    return(df)
    

# B - Test of the code

In [10]:
# testing cell

N = 10
mean_degree = 4
rate_random = 0.005
contagion = 1
beta = 0.2
phi = 0.25
rate_neighbors = 0.05
nber_sample = 10

df = make_propagation(contagion, beta, phi, N, mean_degree, rate_random, rate_neighbors, nber_sample)


# C - Make the whole data set

In [6]:
# parameters

N = 1000
mean_degree = 4
rate_random = 0.005
contagion = 1
beta = 0.2
phi = 0.25
rate_neighbors = 0.05
nber_sample = 10000

In [ ]:
contagion = 0
for beta in [0.1, 0.3, 0.5, 0.7, 0.9]:
    print(beta)
    make_propagation(contagion, beta, phi, N, mean_degree, rate_random, rate_neighbors, nber_sample)

contagion = 1
for phi in [0.1, 0.3, 0.5, 0.7, 0.9]:
    print(phi)
    make_propagation(contagion, beta, phi, N, mean_degree, rate_random, rate_neighbors, nber_sample)


In [ ]:
# end of the code